# セルオートマトン

セルオートマトン（cellular automaton）とは、格子状に広がるセルを用いた離散的計算モデルである。各セルは有限種類の内部状態を持っている。時刻$t$（離散的なので世代とも呼ばれる）でのセルの状態は、一定の規則によって、前時刻での自分自身や周りのセルの状態から決定される<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)。単純化された数理モデルではあるが、複雑で非自明な結果を与え得ることが知られている。

## 今回のねらい

- セルオートマトンのような簡単な離散的計算モデルのシミュレーション方法を理解する。
- 単純な規則から複雑な挙動が創発される例を学ぶ。

## 基本セルオートマトン

もっとも簡単なセルオートマトンとして、1次元にセルが並ぶようなものを考える。各セルは2つの状態（0か1）をとり、自分自身と両側に接している隣の（合わせて3つの）セルの状態によって、次の時刻の状態が決まるものとする。このようなセルオートマトンは、基本セルオートマトン（elementary cellular automaton）と呼ばれる。

最近傍3つのセルの状態の組み合わせは$2^3 = 8$パターンある。各パターンによって中心セルがどのように変化するかを決めることで規則が定義できる。たとえば、図1のように中心セルの状態が変化する規則を考えよう。

![ルール 250](images/rule250.png)

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

図1. ルール250によるセルの遷移規則

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length,ja-engineering-paper/prh -->

最近傍3つのセル8パターンの順番をこのように固定したとき（最近傍3つのセルを2進法表記だとして読むと、右から順に$000_2 = 0$、$001_2 = 1$、$010_2 = 2$、$011_2 = 3$、$\dots$、一番左が$111_2 = 7$である）、この規則をウルフラムの命名法にしたがってルール250と呼ぶ。なぜなら、$11111010_2 = 250$だからである。このような規則は、ルール0（$00000000_2 = 0$）からルール255（$11111111_2 = 255$）までの$2^8 = 256$種類がある。

<!-- textlint-enable -->

このような基本セルオートマトンを実行するプログラムを考えよう。計算機上で無限に長い1次元セルを表現することは困難であるので、セルの個数は有限とし、両端のセルはつながっているものとする（周期境界条件）。すると、1次元に並んだセルは、$0$か$1$の要素が入ったリストによって表せるだろう。セルの初期状態について、とりあえずは、確率$1/2$でランダムに$0$か$1$を設定しよう<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)。次の世代のセルの状態は、上述した規則によって、前の世代のセルの状態から決定する。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random


def random_initialize(size):
    # 長さsizeのランダムに0か1で初期化されたリストを返す関数
    cells = [0] * size
    for i in range(size):
        if random.random() < 0.5:
            cells[i] = 1
    return cells


def cellular_automaton(
    rule, initialize=random_initialize, size=100, tmax=200, figscale=0.08
):
    # セルオートマトンを実行して描画する関数
    #
    # rule: ルールを表すリスト（長さ8、各要素は0か1）
    # initialize: 初期化されたセルをリストで与える関数
    # size: セルのサイズ
    # tmax: 最終時刻
    # figscale: 描画する図のサイズを調節するスケーリング因子

    # 各世代のセルの状態を保存するリスト
    generations = []

    # 初期状態
    cells = initialize(size)
    generations.append(cells)
    size = len(cells)  # 初期化関数が返してきたリストのサイズ（sizeと違ってもよい）を採用する

    for _ in range(tmax):
        next_cells = [0] * size  # 次の世代のセルを新しいリストとして用意する

        for i in range(size):
            # 周期境界条件（leftとrightは必ず0からsize-1の間の値となる）
            left = i - 1
            right = i + 1
            if left < 0:
                left += size
            if right >= size:
                right -= size

            # 最近傍3つのセルの状態を2進法表記とみなして読み取る（kは0から7となる）
            k = cells[left] * 4 + cells[i] * 2 + cells[right]
            # 対応する状態をルールから読み取る
            next_cells[i] = rule[k]

        # セルを更新
        cells = next_cells
        generations.append(cells)

    # 描画する
    fig, ax = plt.subplots(figsize=(size * figscale, tmax * figscale))  # 図のサイズを調節している
    ax.matshow(generations, vmin=0, vmax=1, cmap="viridis")
    plt.show()

上のセルを実行して`cellular_automaton`関数を定義したら、次のセルを実行しよう。

In [ ]:
# ルール250の基本セルオートマトン
cellular_automaton(rule=[0, 1, 0, 1, 1, 1, 1, 1], tmax=50)

<!-- textlint-disable ja-technical-writing/sentence-length -->

例題プログラムの`cellular_automaton`関数に渡す`rule`という引数は、`rule[0]`がセルの状態$000_2$に対する中央セルの次世代での値、`rule[1]`が $001_2$の値、`rule[2]`が$010_2$の値、$\dots$、`rule[7]`が$111_2$の値となるようなリストである。よって、ルール250（$=11111010_2$）に対しては `[0, 1, 0, 1, 1, 1, 1, 1]` となる（リストとして`[...]`のように書くと、左右逆となることに注意）。

<!-- textlint-enable -->

`cellular_automaton`関数では、各世代のセルの状態（`cells`）をすべて`generations`というリストに保存している。各世代のセルの状態はそれ自体が（1次元の）リストであるので、`generations`は言うなれば2次元のリストとなっている。これを[Axesオブジェクト](https://matplotlib.org/stable/api/axes_api.html#the-axes-class)の[matshowメソッド](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.matshow.html)に渡すことで、2次元リストに格納された内容を描画している。縦軸が時間軸である。暗い部分は$0$に、明るい部分は$1$に対応する。

<!-- textlint-disable ja-technical-writing/sentence-length,ja-engineering-paper/prh,ja-technical-writing/ja-no-weak-phrase -->

1980年代、スティーブン・ウルフラムは、今回取り上げる基本セルオートマトン（elementary cellular automaton）の研究から、セルオートマトンをその振る舞いによって4つのクラスに分類している<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3)：
- クラス1：すべてのセルが一様な状態へと収束する（すべて生きている、すべて死ぬなど）<a name="cite_ref-4"></a>[<sup>[4]</sup>](#cite_note-4)。
- クラス2：単純な挙動の1つへと落ち着く（ずっと生き続ける、ずっと死んでいる、周期的に移り変わるなど）。
- クラス3：ランダムな振る舞いを見せる。一時的に局所的な構造が作られるかもしれないが、すぐに露と消える。
- クラス4：秩序状態とランダムな状態の混合である。局所的な構造が移動しながら相互作用を起こす。

<!-- textlint-enable -->

実行すると分かるように、ルール250はすぐ一様な状態（すべてのセルが$1$）に収束し、クラス1に分類される。

## 練習問題

(1) 次にあげるルールに基づく基本セルオートマトンを実行し、クラス2、3、4の特徴も確認しよう<a name="cite_ref-5"></a>[<sup>[5]</sup>](#cite_note-5)。
- ルール4：クラス2の「周期的」なパターンの例。
- ルール30：クラス3の「ランダム」なパターンの例<a name="cite_ref-6"></a>[<sup>[6]</sup>](#cite_note-6)。
- ルール110：クラス4の「複雑な局所構造」を持つパターンの例<a name="cite_ref-7"></a>[<sup>[7]</sup>](#cite_note-7)。

## ルール110のグライダー

練習問題の結果からもわかるように、ルール110では複雑な局所構造が現れ、クラス4に分類される。具体的には、形を保ちながら時間とともに移動する構造が見られるであろう。そのような構造のことをグライダー（glider）と呼ぶことがある<a name="cite_ref-8"></a>[<sup>[8]</sup>](#cite_note-8)。今度は、セルの初期値をランダムに決めるのではなくて、あらかじめ構造を配置したうえでルール110のセルオートマトンを実行してみよう。

In [ ]:
def rule_110_initialize(size):
    # 背景パターン
    background = [0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1]

    # 右に動くグライダー（速度 2/3 ~= 0.666667）
    gliderA4 = [0, 0, 0, 1, 1, 1, 0, 1, 1, 1]

    # 左に動くグライダー（速度 -1/2 = -0.5）
    gliderBbar = [0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1]

    # 動かない物体（速度 0）
    stillC1 = [0, 0, 0, 1, 0, 0, 0, 1, 1]

    # 右に動くグライダー（速度 1/5 = 0.2）
    gliderD1 = [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1]

    # 左に動くグライダー（速度 -4/15 ~= -0.266667）
    gliderE1 = [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1]

    # sizeは無視して好きなものを返す
    return background * 5 + gliderA4 + background * 5


cellular_automaton(
    [0, 1, 1, 1, 0, 1, 1, 0], initialize=rule_110_initialize, tmax=500, figscale=0.04
)

ルール110の基本セルオートマトンは、[チューリング完全（Turing-complete）](https://ja.wikipedia.org/wiki/%E3%83%81%E3%83%A5%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0%E5%AE%8C%E5%85%A8)であることを数学者が証明するのに十分な複雑さを持っている。つまり、現在のデジタル・コンピューターでシミュレーションできることは（原理的には）ルール110の基本セルオートマトンを使ってすべてシミュレーションできる<a name="cite_ref-9"></a>[<sup>[9]</sup>](#cite_note-9)。

## 練習問題

(2) 上のプログラムを、`gliderA4`の代わりにほかの構造を配置するように書き換えて、実行してみよう<a name="cite_ref-10"></a>[<sup>[10]</sup>](#cite_note-10)。

(3) 移動する物体があれば、それを何かにぶつけてみたくなるのが人情というものである。グライダーと静止物体の衝突、グライダーどうしの衝突（正面衝突および追突）をさせてみよう。

## 演習課題

上の例題で定義された`random_initialize`と`cellular_automaton`について、**そのまま変更せず使うのなら**解答セルにそのコピーを書かなくてもよいこととする。ただし、一部でも変更するのであれば解答セル内に定義し直すこと。解答セル内に定義がなければ、上の例題での定義であるとみなす。

<!-- textlint-disable jtf-style/4.3.1.丸かっこ（）,ja-technical-writing/ja-no-redundant-expression -->

(1) 例題のようにランダムな初期配置で、次のルールの基本セルオートマトン(a)から(d)を実行せよ（それぞれについて、どちらを選ぶべきかは**授業中に指定**する）。また、それらをクラス1から4に分類せよ。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

(a) ルール44、あるいはルール100

(b) ルール136、あるいはルール192

(c) ルール54、あるいはルール147

(d) ルール45、あるいはルール101

<!-- textlint-enable -->

In [ ]:
# 課題解答11.1  <-- 提出する際に、この行を必ず含めること。

# ここで(a)から(d)のルールで順番に基本セルオートマトンを実行してください。
# （ルール250になっているので、適切に書き換えること。）

cellular_automaton(rule=[0, 1, 0, 1, 1, 1, 1, 1])  # (a) ルール250
cellular_automaton(rule=[0, 1, 0, 1, 1, 1, 1, 1])  # (b) ルール250
cellular_automaton(rule=[0, 1, 0, 1, 1, 1, 1, 1])  # (c) ルール250
cellular_automaton(rule=[0, 1, 0, 1, 1, 1, 1, 1])  # (d) ルール250

# ここに（Pythonのコメントとして）分類を答えてください。
#
# (a) クラス?
# (b) クラス?
# (c) クラス?
# (d) クラス?

<!-- textlint-disable ja-technical-writing/ja-no-redundant-expression,ja-engineering-paper/prh -->

(2) 初期配置として（ほぼ）中心のセル`cells[size // 2]`のみに$1$を置き、あとはすべて$0$となるようにして、ルール90の基本セルオートマトンを実行せよ。`size`は（デフォルト値である）`100`とする。

<!-- textlint-enable -->

In [ ]:
# 課題解答11.2  <-- 提出する際に、この行を必ず含めること。

# このセルに解答（この行は消してよい）

## 発展課題

<!-- textlint-disable jtf-style/1.1.1.本文 -->

余裕があればやってみてください。

<!-- textlint-enable -->

(3) $0$から$255$までの整数$n$を入力として、今回使った`cellular_automaton`関数の`rule`引数に渡せるようなリストを返す関数`make_rule`を書け。

In [ ]:
# 課題解答11.3  <-- 提出する際に、この行を必ず含めること。


def make_rule(n):  # ルールnの遷移規則を表すリストを返す関数
    ...  # いい感じに書き換えてください。


# ルール250の基本セルオートマトン
cellular_automaton(rule=make_rule(250))

## 脚注

<a name="cite_note-1"></a>1.&nbsp;[^](#cite_ref-1)
一般には、格子は何次元でもよく、また正方格子である必要もない。たとえば、格子の広がる空間がトーラス（ドーナツ状）でもよい。さらに「周りのセル」の定義もさまざまである。

<!-- textlint-disable ja-technical-writing/sentence-length -->

<a name="cite_note-2"></a>2.&nbsp;[^](#cite_ref-2)
例題プログラムの`random_initialize`関数は、[`random.randint`関数](https://docs.python.org/ja/3/library/random.html#random.randint)とリストの内包表記を使えばもっと簡単に書ける（しかし、このように書いておいたほうが、演習課題11.2で何をするべきか分かりやすいのではないだろうか）。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length -->

<a name="cite_note-3"></a>3.&nbsp;[^](#cite_ref-3)
Stephen Wolfram, *A New Kind of Science*, Wolfram Media Inc. (2002) ISBN 978-1-57955-008-0, [[open access]](https://www.wolframscience.com/nks/).

<!-- textlint-enable -->

<a name="cite_note-4"></a>4.&nbsp;[^](#cite_ref-4)
各セルが0か1のどちらかの状態をとるとき、それぞれ「死んでいる」「生きている」と表現することがある。

<a name="cite_note-5"></a>5.&nbsp;[^](#cite_ref-5)
可能ならば、10進法で0から255の数字を、2進法でのリストの表現に直すような関数をあらかじめ作っておくと楽である（今回の発展課題を参照）。

<a name="cite_note-6"></a>6.&nbsp;[^](#cite_ref-6)
ある種の貝の貝殻に、[ルール30に似た模様](https://commons.wikimedia.org/wiki/File:Textile_cone.JPG)の現れることが知られている。

<a name="cite_note-7"></a>7.&nbsp;[^](#cite_ref-7)
局所構造やその相互作用については、`size`や`tmax`を大きくしてみると分かりやすいだろう。

<a name="cite_note-8"></a>8.&nbsp;[^](#cite_ref-8)
もともと、2次元セルオートマトンの1つである[ライフゲーム](https://ja.wikipedia.org/wiki/%E3%83%A9%E3%82%A4%E3%83%95%E3%82%B2%E3%83%BC%E3%83%A0)（Conway's Game of Life）での[移動物体](https://ja.wikipedia.org/wiki/%E3%82%B0%E3%83%A9%E3%82%A4%E3%83%80%E3%83%BC_(%E3%83%A9%E3%82%A4%E3%83%95%E3%82%B2%E3%83%BC%E3%83%A0))に付けられた名前である。「宇宙船（spaceship）」と呼ばれるものの一種である。

<!-- textlint-disable ja-technical-writing/sentence-length,ja-engineering-paper/prh,ja-technical-writing/ja-no-weak-phrase,ja-technical-writing/max-ten -->

<a name="cite_note-9"></a>9.&nbsp;[^](#cite_ref-9)
ルール110基本セルオートマトンやライフゲームなどでは、複雑な多種多様の挙動が観察されるが、それを生み出す規則は驚くほど単純である。ところで、単純で美しい法則に支配されている（少なくともそうであるべきと信じられている）が、複雑な挙動を見せる系がある。それは我々の住んでいる宇宙である。仮に、とてつもなく広大なセル空間を持った3次元のセルオートマトンを考えてみると、それはすでに一種の「宇宙」であると言える。この考えを延長していくと、我々の住んでいる宇宙も本質的には巨大なデジタル・コンピューターの中で実行されているセルオートマトンやシミュレーションなのではないかと考える[シミュレーション仮説（simulation hypothesis）](https://ja.wikipedia.org/wiki/%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E4%BB%AE%E8%AA%AC)や[デジタル物理学（digital physics）](https://ja.wikipedia.org/wiki/%E3%83%87%E3%82%B8%E3%82%BF%E3%83%AB%E7%89%A9%E7%90%86%E5%AD%A6)へと辿り着く（物理というよりもすでに「哲学」であるが）。「空間や時間が飛び飛びであるなんてこと経験したことがない」と反論したくなるかもしれないが、空間や時間の最小単位は我々には小さすぎて認識できず、連続的だと知覚しているだけなのかもしれない。現に、我々は量子力学という「飛び飛びの観測量」が出てくる理論をすでに知っている。さらに、この宇宙が巨大なデジタル・コンピューターだとして、そのコンピューターが我々の知っているような姿をしたものである必要はない。デジタル・コンピューターで実行できることは、ライフゲームやルール110基本セルオートマトンですべてシミュレーションできるのだから。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/sentence-length,ja-engineering-paper/prh,ja-technical-writing/ja-no-weak-phrase,ja-technical-writing/max-ten -->

「その考えが正しいとすると、ひょっとして、自分の存在なんて「実体」を持たないルール110基本セルオートマトンの中のグライダーの集まりだったりするのか」と嘆きたくなるかもしれないが、その必要はまったくない。まず、本当にこの世界がオートマトン上に構築されたシミュレーションだったとしても、それを真の「現実」と区別する手段は（今のところ）ない。区別できないのだから（[世界五分前仮説](https://ja.wikipedia.org/wiki/%E4%B8%96%E7%95%8C%E4%BA%94%E5%88%86%E5%89%8D%E4%BB%AE%E8%AA%AC)と同じようなもので）気にかける必要はない。また、我々がこの世界で観測している対称性の中には、デジタル物理学のモデルと相容れないものがある。たとえば、現在の素粒子物理学の[標準模型](https://ja.wikipedia.org/wiki/%E6%A8%99%E6%BA%96%E6%A8%A1%E5%9E%8B)は、離散的なシミュレーションと非常に相性が悪い（[ニールセン＝二宮の定理](https://ja.wikipedia.org/wiki/%E3%83%95%E3%82%A7%E3%83%AB%E3%83%9F%E3%82%AA%E3%83%B3%E3%83%BB%E3%83%80%E3%83%96%E3%83%AA%E3%83%B3%E3%82%B0#%E3%83%8B%E3%83%BC%E3%83%AB%E3%82%BB%E3%83%B3%EF%BC%9D%E4%BA%8C%E5%AE%AE%E3%81%AE%E5%AE%9A%E7%90%86)）。古典的なセルオートマトンと量子力学はどうしても相容れなさそうだが、暇があるときにいろいろと考えてみるのは面白いかもしれない（なお、筆者にはそんな暇はない）。

<!-- textlint-enable -->

<!-- textlint-disable
ja-technical-writing/max-comma,
ja-technical-writing/sentence-length
-->

- Stephen Wolfram, *A Class of Models with the Potential to Represent Fundamental Physics*, [Complex Syst. 29 (2020) 107-536](https://www.complex-systems.com/abstracts/v29_i02_a01/), [arXiv:2004.08210](https://arxiv.org/abs/2004.08210).
- Gerard t. Hooft, *Ontology in quantum mechanics*, in *Topics on Quantum Information Science*, IntechOpen (2021), [arXiv:2107.14191](https://arxiv.org/abs/2107.14191).

<!-- textlint-enable -->

<a name="cite_note-10"></a>10.&nbsp;[^](#cite_ref-10)
ルール110では、ほかにもさまざまな局所構造が発見されている。いくつかを次にあげる。

```python
    # 背景パターン
    background = [0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1]
    
    # 右に動くグライダー（速度 2/3 ~= 0.666667）
    gliderA = [0, 0, 0, 1, 1, 1]
    
    glider2A = [0, 0, 0, 1, 1, 1] * 2
    
    gliderA2 = [0, 0, 0, 1, 0, 0, 1] + [1, 0, 1, 1, 1]
    
    gliderA4 = [0, 0, 0, 1, 1, 1] + [0, 1, 1, 1]
    
    gliderA5 = [0, 0, 0, 1, 0, 0, 1] + [1, 0, 1, 1, 1] * 2
    
    gliderA7 = [0, 0, 0, 1, 1, 1] + [0, 1, 1, 1] * 2
    
    # 左に動くグライダー（速度 -1/2 = -0.5）
    gliderB = [0, 0, 0, 1, 1, 1, 1, 1]

    glider4B = [0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1]

    gliderBbar = [0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1]

    # 動かない物体（速度 0）
    stillC1 = [0, 0, 0, 1, 0, 0, 0, 1, 1]

    still2C1 = [0, 0, 0, 1, 0, 0, 0, 1, 1] * 2
    
    still3C1 = [0, 0, 0, 1, 0, 0, 0, 1, 1] * 3
    
    stillC2 = [1, 1, 1]
    
    # 右に動くグライダー（速度 1/5 = 0.2）
    gliderD1 = [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1]

    # 左に動くグライダー（速度 -4/15 ~= -0.266667）
    gliderE1 = [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1]

    gliderE3 = [0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

    gliderE4 = [0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1]

    gliderEbar = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]

    # 左に動くグライダー（速度 -1/9 ~= -0.111111）
    gliderF = [0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]

    # 左に動くグライダー（速度 -1/3 ~= -0.333333）
    gliderG = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

    # 左に動くグライダー（速度 -9/46 ~= -0.195652）
    gliderH = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1]

    # グライダー砲（速度 -20/77 ~= -0.25974、グライダーAとBを発射）
    gun = [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1]
```

ここで、構造の命名規則は（できるだけ）次の文献にしたがっている。

- Matthew Cook, *Universality in Elementary Cellular Automata*, [Complex Syst. 15 (2004) 1–40](https://www.complex-systems.com/abstracts/v15_i01_a01/).